In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
seed = 18

merged_resistance_df = pd.read_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/essay_level/merged_themes_essay_level/merged_Resistance_essay_level_batch_1.csv", encoding='utf-8')

# Shuffle the merged dataset
merged_resistance_df = shuffle(merged_resistance_df, random_state=seed)

# Train-test split 
training_df, test_df = train_test_split(merged_resistance_df, test_size=0.2, random_state=18, stratify=merged_resistance_df['label'])

training_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [5]:
print(f"Training dataset shape: {training_df.shape} \nTest dataset shape: {test_df.shape}")
pos_labels = len([n for n in training_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(training_df['label']), (pos_labels/len(training_df['label']))*100))
pos_labels = len([n for n in test_df['label'] if n==1])
print("Positive labels present in the test dataset : {}  out of {} or {}%".format(pos_labels, len(test_df['label']), (pos_labels/len(test_df['label']))*100))

Training dataset shape: (942, 3) 
Test dataset shape: (236, 3)
Positive labels present in the dataset : 118  out of 942 or 12.526539278131635%
Positive labels present in the test dataset : 29  out of 236 or 12.288135593220339%


In [3]:
MAXLEN = 512

In [8]:
X = training_df['essay']
y = training_df['label']

In [9]:
import ktrain
from ktrain import text

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 18, stratify=y)

model_name = 'bert-base-cased'

distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
distillbert_base_model = distillbert_transformer.get_classifier()


Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

preprocessing train...
language: en
train sequence lengths:
	mean : 158
	95percentile : 248
	99percentile : 325


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 155
	95percentile : 253
	99percentile : 322


In [10]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Define classes and class labels
classes = np.array([0, 1])
class_labels = list(training_df.label)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=class_labels)

# Print class weights
print(class_weights)

class_weights = {0:0.56103896,1:4.59574468}

[0.57160194 3.99152542]


In [11]:
import tensorflow as tf
import numpy as np
import os
import random
def reset_random_seeds(seed=2):
    os.environ['PYTHONHASHSEED']=str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [20]:
# Build BERT model
# model = text.text_classifier('distilbert', train_data=(X_train, y_train), preproc=distillbert_transformer)
learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=6)
learner.autofit(2e-5, class_weight=class_weights, early_stopping=3)
# learner.autofit(2.27E-06, early_stopping=4)
# distillbert_learner.autofit(2.27E-06, early_stopping=4, class_weight=class_weights)
# distillbert_learner.set_weight_decay(0.001)
# distillbert_learner.autofit(2.27E-06, early_stopping=4, class_weight=class_weights)

reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 2e-05...
Epoch 1/1024
110/110 [==============================] - 193s 2s/step - loss: 0.7199 - accuracy: 0.6024 - val_loss: 0.5632 - val_accuracy: 0.6961
Epoch 2/1024
110/110 [==============================] - 149s 1s/step - loss: 0.5510 - accuracy: 0.6646 - val_loss: 0.6261 - val_accuracy: 0.6290
Epoch 3/1024
110/110 [==============================] - ETA: 0s - loss: 0.4205 - accuracy: 0.7709
Epoch 00003: Reducing Max LR on Plateau: new max lr will be 1e-05 (if not early_stopping).
110/110 [==============================] - 143s 1s/step - loss: 0.4205 - accuracy: 0.7709 - val_loss: 0.8008 - val_accuracy: 0.6078
Epoch 4/1024
110/110 [==============================] - 139s 1s/step - loss: 0.3257 - accuracy: 0.8376 - val_loss: 0.5925 - val_accuracy: 0.7385
Epoch 4: early stopping
Weights from best epoch have been loaded into model.


In [21]:
learner.validate(class_names=distillbert_transformer.get_classes())

9/9 [==============================] - 12s 1s/step
              precision    recall  f1-score   support

           0       0.90      0.73      0.81       248
           1       0.19      0.43      0.26        35

    accuracy                           0.70       283
   macro avg       0.54      0.58      0.53       283
weighted avg       0.81      0.70      0.74       283



array([[182,  66],
       [ 20,  15]])

In [22]:
distillbert_predictor = ktrain.get_predictor(learner.model, preproc=distillbert_transformer)

distillbert_test_data = test_df['essay'].tolist()
distillbert_test_label = test_df['label'].tolist()

y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

y_pred_distillbert = [int(x) for x in y_pred_distillbert]

tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

True Negative: 145, False Positive: 62, False Negative: 15, True Positive: 14
  Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.70      0.79       207
           1       0.18      0.48      0.27        29

    accuracy                           0.67       236
   macro avg       0.55      0.59      0.53       236
weighted avg       0.82      0.67      0.73       236
 



In [15]:
distillbert_predictor.save('../../../../saved_models/social_bert_base_cased_model_08012024')

: 

In [5]:
learner.validate(class_names=distillbert_transformer.get_classes())

51/51 [==============================] - 29s 427ms/step
              precision    recall  f1-score   support

    negative       0.98      0.95      0.97      1440
    positive       0.71      0.88      0.78       185

    accuracy                           0.94      1625
   macro avg       0.85      0.91      0.88      1625
weighted avg       0.95      0.94      0.95      1625



array([[1373,   67],
       [  23,  162]])